<a href="https://colab.research.google.com/github/amaithi-sam/LLM-Alignment/blob/main/axolotl_instruct_finetune_qlora_openlm_llama_3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/OpenAccess-AI-Collective/axolotl.git

Cloning into 'axolotl'...
remote: Enumerating objects: 13654, done.
remote: Counting objects: 100% (384/384), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 13654 (delta 212), reused 341 (delta 195), pack-reused 13270
Receiving objects: 100% (13654/13654), 5.28 MiB | 16.60 MiB/s, done.
Resolving deltas: 100% (8779/8779), done.


In [3]:
!pwd

/content


In [4]:
%cd axolotl

/content/axolotl


In [6]:
!pip install packaging
!pip install -e .'[flash-attn,deepspedd]'

Obtaining file:///content/axolotl
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/lm-sys/FastChat.git (to revision 27a05b04a35510afb1d767ae7e5990cbd278f8fe) to /tmp/pip-install-1ovoqlv5/fschat_f8a8c703b1ec45c7945df8eef68efee1
  Running command git clone --filter=blob:none --quiet https://github.com/lm-sys/FastChat.git /tmp/pip-install-1ovoqlv5/fschat_f8a8c703b1ec45c7945df8eef68efee1
  Running command git rev-parse -q --verify 'sha^27a05b04a35510afb1d767ae7e5990cbd278f8fe'
  Running command git fetch -q https://github.com/lm-sys/FastChat.git 27a05b04a35510afb1d767ae7e5990cbd278f8fe
  Running command git checkout -q 27a05b04a35510afb1d767ae7e5990cbd278f8fe
  Resolved https://github.com/lm-sys/FastChat.git to commit 27a05b04a35510afb1d767ae7e5990cbd278f8fe
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/hugging

In [1]:
from datasets import load_dataset

dataset = load_dataset("teknium/GPT4-LLM-Cleaned")


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/54568 [00:00<?, ? examples/s]

In [16]:
import datasets
from datasets import load_from_disk

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'input'],
        num_rows: 54568
    })
})

In [8]:
dataset['train'].select(range(500))

Dataset({
    features: ['instruction', 'output', 'input'],
    num_rows: 500
})

In [11]:
dataset = datasets.DatasetDict({'train': dataset['train'].select(range(1000))})

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'input'],
        num_rows: 1000
    })
})

In [13]:
dataset.save_to_disk('teknium/GPT4-LLM-Cleaned_n1000')

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [17]:
data = load_from_disk ('/content/teknium/GPT4-LLM-Cleaned_n1000')

In [18]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'input'],
        num_rows: 1000
    })
})

In [26]:
dataset.push_to_hub('instruction')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/amaithi/instruction/commit/ca1e61952abff31acc4415b71282adb70356eab8', commit_message='Upload dataset', commit_description='', oid='ca1e61952abff31acc4415b71282adb70356eab8', pr_url=None, pr_revision=None, pr_num=None)



```
# QLoRa Configs


base_model: openlm-research/open_llama_3b_v2
model_type: LlamaForCausalLM
tokenizer_type: LlamaTokenizer
load_in_8bit: false
load_in_4bit: true
strict: false
push_dataset_to_hub:
datasets:
  - path: amaithi/instruction
    type: alpaca
dataset_prepared_path:
val_set_size: 0.05
adapter: qlora
lora_model_dir:
sequence_len: 1024
sample_packing: true
lora_r: 8
lora_alpha: 32
lora_dropout: 0.05
lora_target_modules:
  - gate_proj
  - down_proj
  - up_proj
  - q_proj
  - v_proj
  - k_proj
  - o_proj
lora_target_linear: true
lora_fan_in_fan_out:
wandb_project:
wandb_entity:
wandb_watch:
wandb_name:
wandb_log_model:
output_dir: ./outputs/qlora-open_llama_3b-instruct
gradient_accumulation_steps: 1
micro_batch_size: 2
num_epochs: 1
optimizer: paged_adamw_32bit
torchdistx_path:
lr_scheduler: cosine
learning_rate: 0.0002
train_on_inputs: false
group_by_length: false
bf16: true
fp16: false
tf32: false
gradient_checkpointing: true
early_stopping_patience:
resume_from_checkpoint:
local_rank:
logging_steps: 10
xformers_attention:
flash_attention: false
gptq_groupsize:
gptq_model_v1:
warmup_steps: 20
evals_per_epoch: 1
saves_per_epoch: 1
debug:
deepspeed:
weight_decay: 0.1
fsdp:
fsdp_config:
special_tokens:
  bos_token: "<s>"
  eos_token: "</s>"
  unk_token: "<unk>"

```






In [ ]:
! accelerate launch -m axolotl.cli.train /content/axolotl/examples/openllama-3b/qlora.yml

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2024-06-06 07:39:43,654] [INFO] [datasets.<module>:58] [PID:16210] PyTorch version 2.3.0+cu121 available.
[2024-06-06 07:39:43,656] [INFO] [datasets.<module>:70] [PID:16210] Polars version 0.20.2 available.
[2024-06-06 07:39:43,656] [INFO] [datasets.<module>:105] [PID:16210] TensorFlow version 2.15.0 available.
[2024-06-06 07:39:43,657] [INFO] [datasets.<module>:118] [PID:16210] JAX version 0.4.26 available.
2024-06-06 07:39:46.570486: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been regi

In [5]:
import yaml

# YAML file path
yaml_file = '/content/axolotl/examples/openllama-3b/qlora.yml'

# Reading YAML data from file
with open(yaml_file, 'r') as f:
    yaml_data = yaml.load(f, Loader=yaml.FullLoader)

# Displaying parsed YAML data
print(yaml_data)

{'base_model': 'openlm-research/open_llama_3b_v2', 'model_type': 'LlamaForCausalLM', 'tokenizer_type': 'LlamaTokenizer', 'load_in_8bit': False, 'load_in_4bit': True, 'strict': False, 'push_dataset_to_hub': None, 'datasets': [{'path': 'amaithi/instruction', 'type': 'alpaca'}], 'dataset_prepared_path': None, 'val_set_size': 0.05, 'adapter': 'qlora', 'lora_model_dir': None, 'sequence_len': 1024, 'sample_packing': True, 'lora_r': 8, 'lora_alpha': 32, 'lora_dropout': 0.05, 'lora_target_modules': ['gate_proj', 'down_proj', 'up_proj', 'q_proj', 'v_proj', 'k_proj', 'o_proj'], 'lora_target_linear': True, 'lora_fan_in_fan_out': None, 'wandb_project': None, 'wandb_entity': None, 'wandb_watch': None, 'wandb_name': None, 'wandb_log_model': None, 'output_dir': './outputs/qlora-open_llama_3b-instruct', 'gradient_accumulation_steps': 1, 'micro_batch_size': 2, 'num_epochs': 1, 'optimizer': 'paged_adamw_32bit', 'torchdistx_path': None, 'lr_scheduler': 'cosine', 'learning_rate': 0.0002, 'train_on_inputs'

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

config = PeftConfig.from_pretrained("amaithi/qlora-openlm_llama-3b-fine")

base_model = AutoModelForCausalLM.from_pretrained("openlm-research/open_llama_3b_v2")

model = PeftModel.from_pretrained(base_model, "amaithi/qlora-openlm_llama-3b-fine", device_map="auto")

In [ ]:
model = model.merge_and_unload()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openlm-research/open_llama_3b_v2")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("amaithi/qlora-openlm_llama-3b-fine-peft-merged", use_temp_dir=False)
tokenizer.push_to_hub("amaithi/qlora-openlm_llama-3b-fine-peft-merged", use_temp_dir=False)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="amaithi/qlora-openlm_llama-3b-fine-peft-merged")